In [1]:
import anvil.server
import anvil.mpl_util
anvil.server.connect("365ETCRLV4W7EOTU7G4H3XWB-TIQDV4JV6KKM7YQO")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [2]:
from pip import main
import pandas as pd
import requests
import json
import sys
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

C:\Users\shard\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


# MKDIR

In [3]:
pwd = ""
def mkdir(file_name):
  json_object = '{"'+file_name+'":""}'
  r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com/.json')
  dic = json.loads(r1.text)
  if (file_name in dic.keys()):
    return("File already exists")
  else:
    r2 = requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com/.json'.format(pwd),json_object)

def mkdiring(location,file_name):
  prev_loc=""
  for i in location:
        prev_loc = prev_loc+"/"+i
  json_object = '{"'+prev_loc+'/'+file_name+'":""}'
  r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com/.json')
  dic = json.loads(r1.text)
  if (file_name in dic.keys()):
    return("File already exists")
  else:
    r2 = requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com/.json'.format(pwd),json_object)
  

# LS

In [4]:
def ls(file_name):
    r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com/'+file_name+'.json')
    dic = json.loads(r1.text)
    print(dic)
    str1 = ""
    return (str1.join(dic.keys()))
  
def lsing(location):
    prev_loc=""
    for i in location:
        prev_loc = prev_loc+"/"+i
    r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+'.json')
    dic = json.loads(r1.text)
    print(dic)
    str1 = " "
    return (str1.join(dic.keys()))

# CD

In [5]:
def cd(location,file_name):
  global pwd
  if file_name == "..": 
      location.pop()
      if len(location)==0:
        location.append("main")
        str1 = '/'.join(location)
        return str1
      else:
        str1 = '/'.join(location)
        return str1
  else:
      prev_loc = ""
      for i in location:
        prev_loc = prev_loc+"/"+i
      r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+'.json')
      dic = json.loads(r1.text)
      if file_name in dic:
        location.append(file_name)
        str1 = '/'.join(location)
        return str1
        
      else:
        print("Location does not exist")

# RM

In [6]:
def rm(file_name,location):
    json_object = '{"'+file_name+'":""}'
    prev_loc = ""
    for i in location:
        prev_loc = prev_loc+"/"+i
    r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+'.json')
    dic = json.loads(r1.text)
    partition_locs = getPartitions(file_name,location)
    for i in partition_locs:
        r2 = requests.delete(i)
    r3 = requests.delete('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+"/"+file_name+'.json')
    if(r2.status_code!=200):
        return("File doesn't exists")
    elif(r2.status_code==200):
        return("File deleted!")

# CAT

In [7]:
def cat(file_name,location):
    prev_loc = ""
    df = []
    for i in location:
        prev_loc = prev_loc+"/"+i
    location_list=getPartitions(file_name,location)
    for i in range(1,len(location_list)+1):
        df.append(readPartitions(file_name,location,i))
    maindf = pd.concat(df)
    return maindf.dropna()

# PUT

In [8]:
def put(file,location,k):
  df = pd.read_csv(file)
  files = file.split(".")
  file = files[0]
  count_row = df.shape[0]
  k = int(k)
  sliceval = round(count_row/k)
  partitions_df=[]
  z=0
  for i in range (0,k):
    partitions_df.append(df[z:z+sliceval])
    z=z+sliceval
  file_location = []
  for i in range (0,len(partitions_df)):
    js = partitions_df[i].to_json()
    diction = json.loads(js)
    jstring = json.dumps(diction)
    r1 = requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/'+str(i+1)+'/'+file+str(i+1)+'.json',jstring)
    file_location.append('https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/'+str(i+1)+'/'+file+str(i+1)+'.json')


  prev_loc = ""
  metadiction = {"File Name":file,"Number of Partitions":k,"Locations":file_location}
  metajstring = json.dumps(metadiction)
  for i in location:
    prev_loc = prev_loc+"/"+i
  r2 = requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+'/'+file+'.json',metajstring)

# getPartitions

In [9]:
def getPartitions(file_name,location):
  json_object = '{"'+file_name+'":""}'
  prev_loc = ""
  for i in location:
    prev_loc = prev_loc+"/"+i
  r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com'+prev_loc+"/"+file_name+'.json')
  dic = json.loads(r1.text)
  location_list=[]
  location_list=dic["Locations"]
  return location_list


# readPartitions

In [10]:
def readPartitions(file_name,location,kth):
  json_object = '{"'+file_name+'":""}'
  prev_loc = ""
  kth = int(kth)
  for i in location:
    prev_loc = prev_loc+"/"+i
  location_list = getPartitions(file_name,location)
  if(kth <= len(location_list)):
    r1 = requests.get(location_list[kth-1])
    dic = json.loads(r1.text)
    df=pd.DataFrame(dic)
    return df.dropna()
  else:
    return("Required partition does not exist!!!")

# Analytics - Average Temperature

In [11]:
def CityTemp(city, glt):
    citytemps = glt[glt['City'] == city]['AverageTemperature'].dropna()
    return np.average(citytemps)

def avgtemp(location,city):
    parts = getPartitions("CityTemp",location)
    ran = len(parts)
    result = []
    for i in range(1, ran+1):
        glt = readPartitions("CityTemp" , location , i)
        if city in glt['City'].tolist():
            result.append(CityTemp(city, glt))
    return('Average temperature of the city is '+str(round(np.mean(result),2))+' Celsius')

# Analytics - City graph

In [12]:
def citygraph(location,city, start_year, end_year):
    parts = getPartitions("CityTemp",location)
    ran = len(parts)
    index=0
    for i in range(1, ran+1):
        glt = readPartitions("CityTemp" , location , i)
        glt = glt.reset_index().drop('index', axis = 1)
        for j in range (0, len(glt)):
            if glt['City'][j] == city:
                if glt['Years'][j] == start_year:
                    si = index
                if glt['Years'][j] == end_year:
                    ei = index
            index=index+1
    df = cat("CityTemp",location)
    sns.set(rc={'figure.figsize':(40,20)})
    sns.set(font_scale=5)
    ax = sns.lineplot(data= df["AverageTemperature"][si:ei])
    ax.set_xticks([])
    ax.set(xlabel=str(start_year) + '-' + str(end_year),
            ylabel='Temperature in Celcius',
            title='Yearwise City Temperature from ' + str(start_year) + '-' + str(end_year))
    return anvil.mpl_util.plot_image()

# Analytics - Ozone

In [13]:
def ozograph(location, start_year, end_year):
    parts = getPartitions("OzoneHole",location)
    ran = len(parts)
    start_year = int(start_year)
    end_year = int(end_year)
    index = 0
    for i in range(1, ran+1):
        ozo = readPartitions("OzoneHole" , location , i)
        for i in range (0, len(ozo)):
            if start_year == ozo['Year'][index]:
                si = index
            if end_year == ozo['Year'][index]:
                ei = index
            index = index + 1
    temu = cat("OzoneHole",location)
    temu = cat("OzoneHole",location).dropna()
    sns.set(rc={'figure.figsize':(40,20)})
    sns.set(font_scale=5)
    ax = sns.lineplot(data= temu.iloc[si:ei],x="Year", y="Hole Area")
    ax.set_xticks([])
    ax.set(xlabel=str(start_year) + '-' + str(end_year),
            ylabel='Hole Area',
            title='Yearwise Ozone Hole size from ' + str(start_year) + '-' + str(end_year))
    return anvil.mpl_util.plot_image()

# Analytics - Sea Levels

In [14]:
def maxyearsea(location,start_year, end_year):
    parts = getPartitions("Seas",location)
    ran = len(parts)
    temps = []
    index = 0
    startflag = 0
    endflag = 0
    outputs = []
    opindex = 0
    for i in range(1, ran+1):
        tempop = []
        sea = readPartitions("Seas" , location , i).reset_index()
        for s in range (0, len(sea)):
            
            if int(start_year) == int(sea['year'][s]):
                si = index
                startflag = 1
            if int(end_year) == int(sea['year'][s]):
                ei = index
                endflag = 1
            index = index+1
            if startflag == 1 and endflag == 0:
                opindex = i
                last = sea
                tempop.append(pd.DataFrame(sea.iloc[s]))
        outputs.append(tempop)
          
    temu = cat("Seas",location)
    sns.set(rc={'figure.figsize':(40,20)})
    sns.set(font_scale=5)
    ax=sns.lineplot(x = temu['year'][si:ei] , y = temu['total'][si:ei],color='darkblue')
    ax.set(ylabel='Sea Level',
            title='Yearwise Sea Levels ' + str(start_year) + '-' + str(end_year))
    op = pd.concat(outputs[opindex-1], axis=1).T.drop(['index', 'Unnamed: 0'], axis = 1)
    last = last.drop(['index', 'Unnamed: 0'], axis = 1)
    return anvil.mpl_util.plot_image()

# Search - Ozone

In [15]:
def mapsearchOzone(limit, ozo):
    parts = getPartitions("OzoneHole",location)
    ran = len(parts)
    return ozo[ozo['Hole Area'] >= int(limit)]

def reduceOzone(limit):
    parts = getPartitions("OzoneHole",location)
    outputs = []
    for i in range (1, len(parts)+1):
        ozo = readPartitions("OzoneHole" , location , i)
        outputs.append(mapsearchOzone(limit, ozo))
    r1=parts
    r2=ozo
    r3=outputs[-1]
    r4=pd.concat(outputs)
    return r1,r2,r3,r4

# Search - Sea

In [16]:
def mapsearchSea(start_year, end_year, sea):
    return(pd.merge(sea[sea['year'] <= end_year], sea[sea['year'] >= start_year], how ='inner', on =['year', 'total']))

def reduceSea(start_year, end_year):
    parts = getPartitions("Seas",location)
    outputs = []
    for i in range (1, len(parts)+1):
        sea = readPartitions("Seas" , location , i).drop('Unnamed: 0', axis = 1)
        outputs.append(mapsearchSea(start_year, end_year,sea))
    return parts,sea,outputs[-1],pd.concat(outputs)

# Search - Country Temperatures

In [17]:
def mapsearchTemp(country, temp):
    return(temp[temp['Country'] == country])

def reduceTemp(country):
    parts = getPartitions("CityTemp",location)
    outputs = []
    for i in range (1, len(parts)+1):
        temp = readPartitions("CityTemp" , location , i)
        outputs.append(mapsearchTemp(country, temp))
    return parts,temp,outputs[-1],pd.concat(outputs).dropna()

# MAIN

In [18]:
def main():
    global location
    r1 = requests.get('https://usctrojan-18644-default-rtdb.firebaseio.com/.json')
    dic = json.loads(r1.text)
    if ("datanode" not in dic.keys()):
      json_object1 = '{"datanode":""}'
      requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com/.json'.format(pwd),json_object1)
    if ("main" not in dic.keys()):
      json_object2 = '{"main":""}'
      requests.patch('https://usctrojan-18644-default-rtdb.firebaseio.com/.json'.format(pwd),json_object2)
    location = []
    location.append("main")
if __name__=="__main__":
     main()

In [19]:
@anvil.server.callable
def maincode(command):
    command = command.split()
    main_command = command[0]
    if main_command == "mkdir":               #MKDIR
        try:
            filename = command[1]
            if filename.find("/")!=-1:
                t=mkdir(str(filename))
                return t
            else:
                t=mkdiring(location,str(filename))
                return t
        except:
            return("No file name")     
    elif main_command == "ls":                  #LS
        try:
            filename = command[1]
            t2=ls(filename)
            return t2
        except:
            t2=lsing(location)
            return t2
    elif main_command == "rm":                  #RM
        try:
            filename = command[1]
            t4 = rm(str(filename),location)
            return t4
        except:
            return("No file name")  
    elif main_command == "cd":                  #CD
        try:
            directoryname = command[1]
            cd(location,str(directoryname))
            
        except:
            return("Error!")
    elif main_command=="cat":                   #CAT
        try:
            filename = command[1]
            t5=cat(str(filename),location)
            t9=t5.to_markdown()
            return t9
        except:
            return("Error!")
    elif main_command=="put":                   #PUT
        try:
            filename = command[1]
            partitions = command [2]
            put(str(filename),location,partitions)
            return ("Success!")
        except:
            return("Error!")
    elif main_command=="getP":                   #getPartitions
        try:
            filename = command[1]
            df = getPartitions(str(filename),location)
            my_str = '\n'. join(df)
            return(my_str)
        except:
            return("Error!")
    elif main_command=="readP":                   #readPartitions
        try:
            filename = command[1]
            partition = command [2]
            df = readPartitions(str(filename),location,partition)
            return(df.to_markdown())
        except:
            return("Error!")
    elif main_command=="AvgTemp":                   #AvgTemp
        try:
            city = command[1]
            res = avgtemp(location,str(city))
            return(res)
        except:
            return("Error!")
    elif main_command=="CityGraph":                   #City Graph
        try:
            city = command[1]
            sy = command[2]
            ey = command[3]
            res = citygraph(location,str(city),int(sy),int(ey))
            return(res)
        except:
            return("Error!")
    elif main_command=="OzoneGraph":                   #Ozone Graph
        try:
            sy = command[1]
            ey = command[2]
            res = ozograph(location,int(sy),int(ey))
            return(res)
        except:
            return("Error!")
    elif main_command=="SeaLevel":                   #Sea Level
        try:
            sy = command[1]
            ey = command[2]
            res = maxyearsea(location,int(sy),int(ey))
            return(res)
        except:
            return("Error!")
    elif main_command=="SearchOzone":                   #Search Ozone
        try:
            limit = command[1]
            r1,r2,r3,r4 = reduceOzone(str(limit))
            r4.reset_index(drop=True, inplace=True)
            return(r1,r2.to_markdown(),r3.to_markdown(),r4.to_markdown())
        except:
            return("Error!")
    elif main_command=="SearchSea":                   #Search Ozone
        try:
            sy = command[1]
            ey = command[2]
            r1,r2,r3,r4 = reduceSea(int(sy),int(ey))
            r4.reset_index(drop=True, inplace=True)
            return(r1,r2.to_markdown(),r3.to_markdown(),r4.to_markdown())
        except:
            return("Error!")
    elif main_command=="SearchCountryTemp":                   #Search Country Temp
        try:
            country = command[1]
            r1,r2,r3,r4 = reduceTemp(str(country))
            r4.reset_index(drop=True, inplace=True)
            return(r1,r2.to_markdown(),r3.to_markdown(),r4.to_markdown())
        except:
            return("Error!")
    else:
        return("Command not found!!!!")
    

@anvil.server.callable
def currLocation():
    mystr='/'.join(location)
    return mystr

In [20]:
@anvil.server.callable           
def tempanalytics(start_year, end_year):
    df = cat(Ozone)
    si = df.loc[df['Year'] == start_year].index[0]
    ei = df.loc[df['Year'] == end_year].index[0]
    sns.set_style("darkgrid")
    sns.lineplot(data= df[si:ei], x="Year", y="Hole Area")
    return anvil.mpl_util.plot_image()

{'desktop': {'games': ''}, 'documents': '', 'files': {'temperatures': {'Salaries': {'File Name': 'Salaries', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/Salaries1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/Salaries2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/Salaries3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/Salaries4.json'], 'Number of Partitions': 4}}, 'test': '', 'weather': {'CityTemp': {'File Name': 'CityTemp', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/CityTemp1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/CityTemp2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/CityTemp3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/CityTemp4.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/5/CityTemp5.json'], 'Number of Partitions': 5}, 'Ozone': {'Fi

{'temperatures': {'Salaries': {'File Name': 'Salaries', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/Salaries1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/Salaries2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/Salaries3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/Salaries4.json'], 'Number of Partitions': 4}}, 'test': '', 'weather': {'CityTemp': {'File Name': 'CityTemp', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/CityTemp1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/CityTemp2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/CityTemp3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/CityTemp4.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/5/CityTemp5.json'], 'Number of Partitions': 5}, 'Ozone': {'File Name': 'Ozone', 'Locations': ['https://usctrojan-1

{'CityTemp': {'File Name': 'CityTemp', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/CityTemp1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/CityTemp2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/CityTemp3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/CityTemp4.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/5/CityTemp5.json'], 'Number of Partitions': 5}, 'Ozone': {'File Name': 'Ozone', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/Ozone1.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/2/Ozone2.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/3/Ozone3.json', 'https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/4/Ozone4.json'], 'Number of Partitions': 4}, 'OzoneHole': {'File Name': 'OzoneHole', 'Locations': ['https://usctrojan-18644-default-rtdb.firebaseio.com/datanode/1/OzoneHol